# Ray.io optimization framework

In [9]:
import numpy as np
import ray.tune as tune
import torch

Ray is a framework to orchestrate HPO search

Ray\[Tune\] is a mature optimization framework compatible with both PyTorch and TensorFlow, with specific integration with Lightning.

Building a deep learning estimator requires to gradually converge to a

Ray produces a lot of logs...

## Understanding the logic behind an HPO framework

Achieving HPO is a 4-step process:
* Making a selection of the HParams you wish to optimize for, and setting the search space (and choosing for each parameter a sampling method.)
* A callback to monitor and automatically report metrics progress during training
* A trials scheduler to kill unpromising HP sets
* A search algorithm used to explore the HP space
* A logger to push values to a possibly remote monitor solution
* A runner to sequentially execute experiments with the set of HP

HPO allows to 

Let's first load all the necessary params

We will implement all of these components 

## Exploring components in detail

## First fully functional example with no HPO

### **Search space**

Each HP has its own space. Ray comes standard with a range of params types. Report 
* `tune.uniform`, `tune.quniform`, and `tune.qloguniform` to uniformly sample a float in a range of values
* `tune.randn`, `tune.qrandn`, `tune.randint`, `tune.qrandint`, `tune.lograndint`, and `tune.qlograndint` to uniformly sample an integer in a range of values
* `tune.choice` to sample from a discrete list of values
* `tune.sample_from` for a custom-made sampling method
* `tune.grid_search` to end-up browsing an entire list sequentially

Create a config `dict` for data and models HPs

In [8]:
config = {
    "data": {
        "batch_size": tune.choice([2 ** k for k in np.arange(4, 8)])
    },
    "model": {
        "in_channels": 20,
        "out_channels": 4,
        "hidden_channels": int(tune.choice([2 ** k for k in np.arange(4, 6)]).sample()),
        "lr": tune.loguniform(1e-4, 1e-1),
        "dropout": tune.uniform(0, 1),
    }
}

In [12]:
hidden_feats = config["model"]["hidden_channels"]
hidden_feats

32

### **Runner**

The overall is a simple 3-step process

1. Sample parameters to make up an HP set, following a specific search algorithm
2. Build an execution stack with desired number of runs, then start at the top
3. Monitor the training on relevant metrics, stop unpromising trainings early, and move the stack with freed-up resources

The runner will execute runs sequentially`trainable` 

trainable = 

In [ ]:
tune.run(trainable, num_samples=10)

# Run 1 trial, stop when trial has reached 10 iterations
tune.run(my_trainable, stop={"training_iteration": 10})

# automatically retry failed trials up to 3 times
tune.run(my_trainable, stop={"training_iteration": 10}, max_failures=3)

# Run 1 trial, search over hyperparameters, stop after 10 iterations.
space = {"lr": tune.uniform(0, 1), "momentum": tune.uniform(0, 1)}
tune.run(my_trainable, config=space, stop={"training_iteration": 10})

# Resumes training if a previous machine crashed
tune.run(my_trainable, config=space,
         local_dir=<path/to/dir>, resume=True)

# Rerun ONLY failed trials after an experiment is finished.
tune.run(my_trainable, config=space,
         local_dir=<path/to/dir>, resume="ERRORED_ONLY")